In [32]:
import pandas as pd
from backtesting import Backtest, Strategy
import math
from vnstock3 import Vnstock

PHẦN NÀY ĐỂ TEST

In [33]:
class DCA(Strategy):

    amount_to_invest = 10

    def init(self):

        self.day_of_week = self.I(
                            lambda x: x,
                            self.data.Close.s.index.dayofweek,
                            plot = False,
                            )

    def next(self):

        if self.day_of_week[-1] == 1:
            self.buy( size = math.floor(self.amount_to_invest / self.data.Close[-1]))
            # try:
            #     if self.data.Close[-1]/self.data.Close[-30] < 0.95:
            #         self.buy(size = math.floor(self.amount_to_invest / self.data.Close[-1]))
            # except:
            #     pass


In [34]:
kdc = Vnstock().stock(symbol='FPT').quote.history(start='2019-01-01', end='2024-01-04')
kdc = kdc.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
# Set 'time' as index and convert to DatetimeIndex
kdc.set_index('time', inplace=True)
kdc.index = pd.to_datetime(kdc.index)
kdc

2024-08-19 13:15:03,386 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


,Open,High,Low,Close,Volume
time,,,,,
2019-01-02,15.58,15.71,15.33,15.35,459120
2019-01-03,15.45,15.45,15.17,15.20,716600
2019-01-04,15.13,15.45,15.09,15.32,318570
2019-01-07,15.58,15.69,15.50,15.58,408470
2019-01-08,15.58,15.65,15.54,15.65,295660
...,...,...,...,...,...
2023-12-27,84.11,84.46,83.68,83.68,1359748
2023-12-28,83.77,83.77,83.16,83.42,1199164
2023-12-29,83.42,83.77,82.99,82.99,1869204


In [35]:
usd_vnd_data = pd.read_csv('D:/datcom lab/BACKTESTING LIBRARY LEARNING/VND=XCommon.csv')
usd_vnd_data['Date'] = pd.to_datetime(usd_vnd_data['Date'])
usd_vnd_data.set_index('Date', inplace=True)
usd_vnd_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,23172.699219,23194.0,23158.500000,23194.0,23194.0,0.0
2019-01-03,23194.000000,23194.0,23159.500000,23194.0,23194.0,0.0
2019-01-04,23194.000000,23194.0,23160.500000,23194.0,23194.0,0.0
2019-01-07,22854.099609,23194.0,22831.699219,23194.0,23194.0,0.0
2019-01-08,22795.000000,23194.0,22795.000000,23194.0,23194.0,0.0
...,...,...,...,...,...,...
2023-12-27,24325.000000,24380.0,24300.000000,24325.0,24325.0,0.0
2023-12-28,24345.000000,24325.0,24235.000000,24345.0,24345.0,0.0
2023-12-29,24235.000000,24265.0,24230.000000,24235.0,24235.0,0.0


In [36]:
kdc['usd/vnd'] = usd_vnd_data['Close']/1000
kdc['Close'] = kdc['Close'] /kdc['usd/vnd']


In [37]:
kdc = kdc.dropna()

In [38]:
bt = Backtest(
        kdc,
        DCA,
        trade_on_close = True,
        )
stats = bt.run()
print(stats)


Start                     2019-01-02 00:00:00
End                       2024-01-03 00:00:00
Duration                   1827 days 00:00:00
Exposure Time [%]                    99.60032
Equity Final [$]                 13074.193682
Equity Peak [$]                  13284.836125
Return [%]                          30.741937
Buy & Hold Return [%]              416.569672
Return (Ann.) [%]                    5.548116
Volatility (Ann.) [%]                6.451963
Sharpe Ratio                         0.859911
Sortino Ratio                        1.340884
Calmar Ratio                         0.465903
Max. Drawdown [%]                  -11.908316
Avg. Drawdown [%]                   -0.742391
Max. Drawdown Duration      468 days 00:00:00
Avg. Drawdown Duration       26 days 00:00:00
# Trades                                  253
Win Rate [%]                        97.628458
Best Trade [%]                     405.947834
Worst Trade [%]                     -3.771879
Avg. Trade [%]                    

In [39]:
trades = stats["_trades"]
price_paid = trades["Size"] * trades["EntryPrice"]
total_invested = price_paid.sum()

current_shares = trades["Size"].sum()
current_equity = current_shares * kdc.Close.iloc[-1]

print("Total investment:",total_invested)
print("Current Shares:",current_shares )
print("current Equity:", current_equity)

print("Return:", current_equity / total_invested)


Total investment: 2282.1369036062315
Current Shares: 1569
current Equity: 5363.948201438849
Return: 2.3504059694940915


PHẦN CODE CHÍNH BÊN DƯỚI

LÔ LẺ VÀ THEO TUẦN

In [51]:
import pandas as pd
from backtesting import Backtest, Strategy
import math
from vnstock3 import Vnstock

class DCA(Strategy):
    average_monthly_income_vnd = 500  # Average monthly income in dollar
    investment_percentage = 0.10  # Percentage of income to invest

    def init(self):
        self.amount_to_invest = self.average_monthly_income_vnd * self.investment_percentage/4
        self.day_of_week = self.I(
            lambda x: x,
            self.data.Close.s.index.dayofweek,
            plot=False,
        )

    def next(self):
        if self.day_of_week[-1] == 1:
            self.buy(size=math.floor(self.amount_to_invest / self.data.Close[-1]))
            # try:
            #     if self.data.Close[-1] / self.data.Close[-30] < 0.95:
            #         self.buy(size=math.floor(self.amount_to_invest / self.data.Close[-1]))
            # except:
            #     pass

def run_backtest(stock_symbol, usd_vnd_data):
    # Fetch stock data
    stock_data = Vnstock().stock(symbol=stock_symbol).quote.history(start='2019-01-01', end='2024-01-04')
    stock_data = stock_data.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
    stock_data.set_index('time', inplace=True)
    stock_data.index = pd.to_datetime(stock_data.index)

    # Merge USD/VND data
    stock_data['usd/vnd'] = usd_vnd_data['Close'] / 1000
    stock_data['Close'] = stock_data['Close'] / stock_data['usd/vnd']

    # Handle NaN values (choose one method)
    stock_data = stock_data.dropna()
    # stock_data = stock_data.interpolate(method='linear')
    # stock_data = stock_data.ffill()
    # stock_data = stock_data.bfill()

    # Run the backtest
    bt = Backtest(
        stock_data,
        DCA,
        trade_on_close=True,
    )
    stats = bt.run()
    #bt.plot(filename=f'{stock_symbol}')
    # Calculate investment details
    trades = stats["_trades"]
    price_paid = trades["Size"] * trades["EntryPrice"]
    total_invested = price_paid.sum()

    current_shares = trades["Size"].sum()
    current_equity = current_shares * stock_data.Close.iloc[-1]

    print(f"Results for {stock_symbol}:")
    print("Total investment:", total_invested)
    print("Current Shares:", current_shares)
    print("Current Equity:", current_equity)
    print("RoR:", ((current_equity - total_invested) / total_invested)*100)
    print("-" * 50)

# Load USD/VND data
usd_vnd_data = pd.read_csv('D:/datcom lab/BACKTESTING LIBRARY LEARNING/VND=XCommon.csv')
usd_vnd_data['Date'] = pd.to_datetime(usd_vnd_data['Date'])
usd_vnd_data.set_index('Date', inplace=True)

# List of stock symbols
stock_symbols = ["VNM", "DPM", "MBB", "MWG", "SSI", 'VRE']  # Add more stock symbols as needed

# Run backtest for each stock
for symbol in stock_symbols:
    run_backtest(symbol, usd_vnd_data)

2024-08-19 13:32:40,838 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


2024-08-19 13:32:41,830 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Results for VNM:
Total investment: 2797.630415723753
Current Shares: 851
Current Equity: 2383.149845837616
RoR: -14.815415487213665
--------------------------------------------------


2024-08-19 13:32:42,766 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Results for DPM:
Total investment: 3041.4668333134273
Current Shares: 4426
Current Equity: 6177.294964028778
RoR: 103.10249305921657
--------------------------------------------------


...

Results for MBB:
Total investment: 3082.2922545303863
Current Shares: 5817
Current Equity: 4481.421582733813
RoR: 45.39249404877073
--------------------------------------------------


2024-08-19 13:32:44,738 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Results for MWG:
Total investment: 2929.9884167160667
Current Shares: 1720
Current Equity: 3030.5940390544706
RoR: 3.4336525620522003
--------------------------------------------------


2024-08-19 13:32:46,714 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Results for SSI:
Total investment: 3057.850949828696
Current Shares: 4728
Current Equity: 6404.423432682426
RoR: 109.44197535335098
--------------------------------------------------
Results for VRE:
Total investment: 3018.6558711504426
Current Shares: 2419
Current Equity: 2312.0966084275437
RoR: -23.406419707378618
--------------------------------------------------


LÔ CHẴN THEO TUẦN

In [52]:
import pandas as pd
from backtesting import Backtest, Strategy
import math
from vnstock3 import Vnstock


class DCA(Strategy):
    average_monthly_income_vnd = 500  # Average monthly income in dollar
    investment_percentage = 0.10  # Percentage of income to invest
    fund = 0
    def init(self):
        self.amount_to_invest = self.average_monthly_income_vnd * self.investment_percentage/4
        self.day_of_week = self.I(
            lambda x: x,
            self.data.Close.s.index.dayofweek,
            plot=False,
        )

    def next(self):
        if self.day_of_week[-1] == 1:
            share_price = self.data.Close[-1]
            shares_to_buy = self.amount_to_invest // share_price
            if shares_to_buy >= 100:
                shares_to_buy = (shares_to_buy // 100) * 100
                self.buy(size=shares_to_buy)
                #print(f"Buy executed at {self.data.index[-1]} with {shares_to_buy} shares at price {share_price}, total price {share_price * shares_to_buy}")
            else:
                self.fund += self.amount_to_invest
                shares_to_buy = self.fund // share_price
                if shares_to_buy >= 100:
                    shares_to_buy = (shares_to_buy // 100) * 100
                    self.fund -= shares_to_buy * share_price
                    self.buy(size=shares_to_buy)
           
def run_backtest(stock_symbol, usd_vnd_data):
    # Fetch stock data
    stock_data = Vnstock().stock(symbol=stock_symbol).quote.history(start='2019-01-01', end='2024-01-04')
    stock_data = stock_data.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
    stock_data.set_index('time', inplace=True)
    stock_data.index = pd.to_datetime(stock_data.index)

    # Merge USD/VND data
    stock_data['usd/vnd'] = usd_vnd_data['Close'] / 1000
    stock_data['Close'] = stock_data['Close'] / stock_data['usd/vnd']

    # Handle NaN values (choose one method)
    stock_data = stock_data.dropna()
    # stock_data = stock_data.interpolate(method='linear')
    # stock_data = stock_data.ffill()
    # stock_data = stock_data.bfill()

    # Run the backtest
    bt = Backtest(
        stock_data,
        DCA,
        trade_on_close=True,
    )
    stats = bt.run()
    #bt.plot(filename=f'{stock_symbol}')
    # Calculate investment details
    trades = stats["_trades"]
    price_paid = trades["Size"] * trades["EntryPrice"]
    total_invested = price_paid.sum()

    current_shares = trades["Size"].sum()
    current_equity = current_shares * stock_data.Close.iloc[-1]

    print(f"Results for {stock_symbol}:")
    print("Total investment:", total_invested)
    print("Current Shares:", current_shares)
    print("Current Equity:", current_equity)
    print("RoR:", ((current_equity - total_invested) / total_invested)*100)
    print("-" * 50)

# Load USD/VND data
usd_vnd_data = pd.read_csv('D:/datcom lab/BACKTESTING LIBRARY LEARNING/VND=XCommon.csv')
usd_vnd_data['Date'] = pd.to_datetime(usd_vnd_data['Date'])
usd_vnd_data.set_index('Date', inplace=True)

# List of stock symbols
stock_symbols = ["VNM", "DPM", "MBB", "MWG", "SSI", 'VRE']  # Add more stock symbols as needed

# Run backtest for each stock
for symbol in stock_symbols:
    run_backtest(symbol, usd_vnd_data)


2024-08-19 13:32:47,807 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


2024-08-19 13:32:48,602 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Results for VNM:
Total investment: 2919.3377550366804
Current Shares: 900
Current Equity: 2520.369989722508
RoR: -13.666379117176172
--------------------------------------------------


2024-08-19 13:32:49,493 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Results for DPM:
Total investment: 3093.4142444040995
Current Shares: 4500
Current Equity: 6280.575539568346
RoR: 103.03053659656909
--------------------------------------------------


...

Results for MBB:
Total investment: 3127.5424603731003
Current Shares: 5900
Current Equity: 4545.364850976362
RoR: 45.33343379242634
--------------------------------------------------


2024-08-19 13:32:51,129 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Results for MWG:
Total investment: 3125.1751847606147
Current Shares: 1800
Current Equity: 3171.551901336074
RoR: 1.4839717402597945
--------------------------------------------------


2024-08-19 13:32:52,023 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Results for SSI:
Total investment: 3082.5408863428083
Current Shares: 4800
Current Equity: 6501.952723535458
RoR: 110.92835304609739
--------------------------------------------------
Results for VRE:
Total investment: 3123.0563897583716
Current Shares: 2500
Current Equity: 2389.5169578622817
RoR: -23.487870225514666
--------------------------------------------------


LÔ CHẴN VÀ THEO THÁNG

In [53]:
import pandas as pd
from backtesting import Backtest, Strategy
import math
from vnstock3 import Vnstock

def calculate_first_mondays(dates):
        if not isinstance(dates, pd.DatetimeIndex):
            dates = pd.DatetimeIndex(dates)
        dates_series = pd.Series(dates, index=dates)
        mondays = dates_series[dates_series.dt.dayofweek == 0]
        first_mondays = mondays.groupby([mondays.dt.year, mondays.dt.month]).first()
        return set(first_mondays)

class DCA(Strategy):
    average_monthly_income_vnd = 500  # Average monthly income in dollar
    investment_percentage = 0.10  # Percentage of income to invest
    fund = 0
    def init(self):
        self.amount_to_invest = self.average_monthly_income_vnd * self.investment_percentage
        self.first_mondays = calculate_first_mondays(self.data.index)
        # self.day_of_week = self.I(
        #     lambda x: x,
        #     self.data.Close.s.index.dayofweek,
        #     plot=False,
        # )

    def next(self):
        today = self.data.index[-1]
        if today in self.first_mondays:
            # self.buy(size=math.floor(self.amount_to_invest / self.data.Close[-1]))
            share_price = self.data.Close[-1]
            shares_to_buy = self.amount_to_invest // share_price
            if shares_to_buy >= 100:
                shares_to_buy = (shares_to_buy // 100) * 100
                self.buy(size=shares_to_buy)
                #print(f"Buy executed at {self.data.index[-1]} with {shares_to_buy} shares at price {share_price}, total price {share_price * shares_to_buy}")
            else:
                self.fund += self.amount_to_invest
                shares_to_buy = self.fund // share_price
                if shares_to_buy >= 100:
                    shares_to_buy = (shares_to_buy // 100) * 100
                    self.fund -= shares_to_buy * share_price
                    self.buy(size=shares_to_buy)
                    #print(self.fund)
                    #print(f"Buy by fund executed at {self.data.index[-1]} with {shares_to_buy} shares at price {share_price}, total price {share_price * shares_to_buy}")
def run_backtest(stock_symbol, usd_vnd_data):
    # Fetch stock data
    stock_data = Vnstock().stock(symbol=stock_symbol).quote.history(start='2019-01-01', end='2024-01-04')
    stock_data = stock_data.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
    stock_data.set_index('time', inplace=True)
    stock_data.index = pd.to_datetime(stock_data.index)

    # Merge USD/VND data
    stock_data['usd/vnd'] = usd_vnd_data['Close'] / 1000
    stock_data['Close'] = stock_data['Close'] / stock_data['usd/vnd']

    # Handle NaN values (choose one method)
    stock_data = stock_data.dropna()
    # stock_data = stock_data.interpolate(method='linear')
    # stock_data = stock_data.ffill()
    # stock_data = stock_data.bfill()

    # Run the backtest
    bt = Backtest(
        stock_data,
        DCA,
        trade_on_close=True,
    )
    stats = bt.run()
    #bt.plot(filename=f'{stock_symbol}')
    # Calculate investment details
    trades = stats["_trades"]
    price_paid = trades["Size"] * trades["EntryPrice"]
    total_invested = price_paid.sum()

    current_shares = trades["Size"].sum()
    current_equity = current_shares * stock_data.Close.iloc[-1]

    print(f"Results for {stock_symbol}:")
    print("Total investment:", total_invested)
    print("Current Shares:", current_shares)
    print("Current Equity:", current_equity)
    print("RoR:", ((current_equity - total_invested) / total_invested)*100)
    print("-" * 50)

# Load USD/VND data
usd_vnd_data = pd.read_csv('D:/datcom lab/BACKTESTING LIBRARY LEARNING/VND=XCommon.csv')
usd_vnd_data['Date'] = pd.to_datetime(usd_vnd_data['Date'])
usd_vnd_data.set_index('Date', inplace=True)

# List of stock symbols
#stock_symbols = ['SSI', 'EIB', 'HPG']  # Add more stock symbols as needed
stock_symbols = ["VNM", "DPM", "MBB", "MWG", "SSI", 'VRE']
# Run backtest for each stock
for symbol in stock_symbols:
    run_backtest(symbol, usd_vnd_data)


2024-08-19 13:32:53,043 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


2024-08-19 13:32:53,849 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Results for VNM:
Total investment: 2906.7042260814187
Current Shares: 900
Current Equity: 2520.369989722508
RoR: -13.291143725336482
--------------------------------------------------


2024-08-19 13:32:54,693 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Results for DPM:
Total investment: 2724.540986912533
Current Shares: 3800
Current Equity: 5303.597122302159
RoR: 94.66020690377754
--------------------------------------------------


...

Results for MBB:
Total investment: 2679.0911186330277
Current Shares: 4800
Current Equity: 3697.92394655704
RoR: 38.029047270473534
--------------------------------------------------


2024-08-19 13:32:56,370 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Results for MWG:
Total investment: 2859.0652804272304
Current Shares: 1700
Current Equity: 2995.35457348407
RoR: 4.7669178451383205
--------------------------------------------------


2024-08-19 13:32:57,227 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Results for SSI:
Total investment: 2681.6805084490425
Current Shares: 4000
Current Equity: 5418.293936279549
RoR: 102.0484513053807
--------------------------------------------------
Results for VRE:
Total investment: 2928.3029450136164
Current Shares: 2300
Current Equity: 2198.3556012332992
RoR: -24.927316520419744
--------------------------------------------------


In [54]:
import random

# Generate a random number between 1 and 30
random_number = random.randint(1, 30)

# Print the random number
print(random_number)


24
